In [ ]:
import re
import time
import sys
import curses
from rich.live import Live
from rich.table import Table
from rich.text import Text
from rich.align import Align
log_path="/home/wuct/ALICE/reps/hf-vn-working/correlations/Distributions/ME/log.txt"
input_file_list="/home/wuct/ALICE/reps/hf-vn-working/correlations/Distributions/ME/input_derived_data.txt"

def get_processed_files(log_path):
    processed_files = []
    partern = r"Opening file: lfn=(.*?),"
    with open(log_path, "r") as file:
        lines = file.readlines()
    for line in lines:
        if "Opening file: lfn=" in line:
            match = re.search(partern, line)
            if match:
                file_path = match.group(1)
                processed_files.append(file_path)
        if "Dumping used configuration in dpl-config.json" in line:
            sys.exit(0)
    return processed_files

def get_input_files(input_file_list):
    input_files = []
    with open(input_file_list, "r") as file:
        lines = file.readlines()
    for line in lines:
        input_files.append(line.strip())
    return input_files

input_files = get_input_files(input_file_list)
processed_files = []
initital_flag = True
with Live(refresh_per_second=60) as live:
    while processed_files != input_files:
        table = Table(box=None)
        table.add_column(f"Speed Monitor (Total Files: {len(input_files)})", justify="center", style="bold magenta")

        speed_text = Text("Speed: NA files/sec", style="cyan")

        if initital_flag:
            star_time = time.time()
            initital_processed_files = get_processed_files(log_path)
            initital_flag = False

        processed_files = get_processed_files(log_path)
        if len(initital_processed_files) != len(processed_files):
            end_time = time.time()
            speed = (len(processed_files) - len(initital_processed_files)) / (end_time - star_time)
            speed_text = Text(f"Speed: {speed:.2f} files/sec", style="cyan")
        unprocessed_files = set(input_files) - set(processed_files)

        table.add_row(Align.left(speed_text))
        table.add_row(Align.left(Text(f"Processed files: {len(processed_files)}", style="green")))
        table.add_row(Align.left(Text(f"Unprocessed files: {len(unprocessed_files)}", style="red")))

        live.update(table)
        time.sleep(60)
